In [1]:
import numpy as np
import pandas as pd
from math import isnan
from datetime import datetime,date
# Saved Tiingo API Token in 'token' variable in apiToken.py file
from apiToken import token

In [2]:
# Getting prices from Tiingo
def getPrice(token,ticker,startDate,endDate):
    return pd.read_csv('https://api.tiingo.com/tiingo/daily/'+ticker+'/prices?startDate='+startDate+'&endDate='+endDate+'&token='+token+'&format=csv')
today = str(date.today())
bndPrice = getPrice(token,'bnd','2019-09-30',today)
bndxPrice = getPrice(token,'bndx','2019-09-30',today)
vtiPrice = getPrice(token,'vti','2019-09-30',today)
vxusPrice = getPrice(token,'vxus','2019-09-30',today)
vtiPrice['returns'] = (vtiPrice.adjClose)/vtiPrice.adjClose.shift(1) - 1
vxusPrice['returns'] = (vxusPrice.adjClose)/vxusPrice.adjClose.shift(1) - 1
bndPrice['returns'] = (bndPrice.adjClose)/bndPrice.adjClose.shift(1) - 1
bndxPrice['returns'] = (bndxPrice.adjClose)/bndxPrice.adjClose.shift(1) - 1
bndPrice = bndPrice.loc[:,['date','returns','adjClose']]
bndxPrice = bndxPrice.loc[:,['date','returns','adjClose']]
vtiPrice = vtiPrice.loc[:,['date','returns','adjClose']]
vxusPrice = vxusPrice.loc[:,['date','returns','adjClose']]

In [3]:
# Reading saved data (Market cap is in billions)
vti = pd.read_csv('Data/vti_MC_Data.csv')
vxus = pd.read_csv('Data/vxus_MC_Data.csv')
bnd = pd.read_csv('Data/bnd_MC_Data.csv')
bndx = pd.read_csv('Data/bndx_MC_Data.csv')

In [4]:
df = vtiPrice.rename(columns={"returns":"VTI_Return","adjClose":"VTI_Price"}).merge(vxusPrice.rename(columns={"returns":"VXUS_Return","adjClose":"VXUS_Price"}),on='date',how='outer').merge(bndPrice.rename(columns={"returns":"BND_Return","adjClose":"BND_Price"}),on='date',how='outer').merge(bndxPrice.rename(columns={"returns":"BNDX_Return","adjClose":"BNDX_Price"}),on='date',how='outer').merge(vti.rename(columns={"Market Cap":"VTI_MarketCap"}),on='date',how='left').merge(vxus.rename(columns={"Market Cap":"VXUS_MarketCap"}),on='date',how='left').merge(bnd.rename(columns={"Market Cap":"BND_MarketCap"}),on='date',how='left').merge(bndx.rename(columns={"Market Cap":"BNDX_MarketCap"}),on='date',how='left').sort_values(by='date').reset_index(drop=True)

In [5]:
etfs = ['VTI','VXUS','BND','BNDX']
for etf in etfs:
    list = []
    i = 0
    value = 999
    for mc,ret in zip(df[etf+'_MarketCap'],df[etf+'_Return']):
        if i>0:
            value = (ret+1) * list[i-1]
        list.append(mc) if not isnan(mc) else list.append(value)
        i+=1
    df[etf+'_MarketCap'] = list

In [6]:
df['VTI_weight'] = df['VTI_MarketCap']/(df.VTI_MarketCap+df.VXUS_MarketCap+df.BND_MarketCap+df.BNDX_MarketCap)
df['VXUS_weight'] = df['VXUS_MarketCap']/(df.VTI_MarketCap+df.VXUS_MarketCap+df.BND_MarketCap+df.BNDX_MarketCap)
df['BND_weight'] = df['BND_MarketCap']/(df.VTI_MarketCap+df.VXUS_MarketCap+df.BND_MarketCap+df.BNDX_MarketCap)
df['BNDX_weight'] = df['BNDX_MarketCap']/(df.VTI_MarketCap+df.VXUS_MarketCap+df.BND_MarketCap+df.BNDX_MarketCap)

#Enter starting amount
startingAmount = 10000

portfolioVal = [0 for i in range(len(df))]
portfolioVal[0] = startingAmount
df['Portfolio_Value'] = portfolioVal

In [7]:
i=0
list2 = []
portfolioRet = 1
for vtiR,vxusR,bndR,bndxR,vtiW,vxusW,bndW,bndxW,portfolioValue in zip(df.VTI_Return,df.VXUS_Return,df.BND_Return,df.BNDX_Return,df.VTI_weight.shift(1),df.VXUS_weight.shift(1),df.BND_weight.shift(1),df.BNDX_weight.shift(1),df.Portfolio_Value):
    if i>0:
        portfolioRet = (vtiR*vtiW+vxusR*vxusW+bndR*bndW+bndxR*bndxW)+1
    list2.append(portfolioValue) if portfolioValue != 0 else list2.append(list2[i-1]*portfolioRet)
    i+=1
df['Portfolio_Value'] = list2

In [8]:
df

,date,VTI_Return,VTI_Price,VXUS_Return,VXUS_Price,BND_Return,BND_Price,BNDX_Return,BNDX_Price,VTI_MarketCap,VXUS_MarketCap,BND_MarketCap,BNDX_MarketCap,VTI_weight,VXUS_weight,BND_weight,BNDX_weight,Portfolio_Value
0,2019-09-30,NaN,148.218599,NaN,50.350477,NaN,82.259751,NaN,56.889932,29401.430000,999.000000,22512.530000,19421.010000,0.406468,0.013811,0.311230,0.268491,10000.000000
1,2019-10-01,-0.013046,146.284886,-0.009098,49.892391,0.000967,82.339308,-0.001668,56.795067,29017.848430,989.911150,22534.302867,19388.625144,0.403414,0.013762,0.313278,0.269546,9944.247162
2,2019-10-02,-0.016238,143.909462,-0.015628,49.112670,0.002253,82.524823,0.000000,56.795067,28546.646704,974.440767,22585.073856,19388.625144,0.399283,0.013630,0.315898,0.271189,9883.984865
3,2019-10-03,0.008117,145.077543,0.007740,49.492784,0.003313,82.798214,0.003749,57.008000,28778.353338,981.982578,22659.894262,19461.315968,0.400358,0.013661,0.315239,0.270741,9937.453474
4,2019-10-04,0.013058,146.971992,0.006696,49.824166,0.001651,82.934909,0.000170,57.017679,29154.146450,988.557491,22697.304465,19464.620096,0.403213,0.013672,0.313912,0.269203,9995.943607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,2020-10-22,0.007085,176.260000,0.000187,53.370000,-0.001710,87.590000,-0.001372,58.210000,34454.004092,25178.515384,23587.329501,22168.892845,0.326923,0.238911,0.223813,0.210354,10877.819935
270,2020-10-23,0.003291,176.840000,0.005621,53.670000,0.001256,87.700000,0.001031,58.270000,34567.378212,25320.047229,23616.951675,22191.743447,0.327045,0.239555,0.223442,0.209958,10909.546385
271,2020-10-26,-0.018152,173.630000,-0.016769,52.770000,0.001482,87.830000,0.000858,58.320000,33939.911100,24895.451692,23651.959699,22210.785616,0.324169,0.237783,0.225906,0.212141,10806.535384
272,2020-10-27,-0.004032,172.930000,-0.003601,52.580000,0.002391,88.040000,0.001886,58.430000,33803.080266,24805.814856,23708.511123,22252.678387,0.323258,0.237217,0.226724,0.212802,10793.321294
